<a href="https://colab.research.google.com/github/YOUR_USERNAME/Digital-Finance-Introduction/blob/main/day_01/notebooks/NB01_Money_Ledgers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NB01: Money and Ledgers

**Topic:** 1.1 - Why Digital Finance: The Foundation of Financial Systems

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand Ledgers**: Explain how ledgers record financial transactions and maintain balances
2. **Build a Simple Ledger**: Implement a working ledger system in Python from scratch
3. **Experience Double-Spending**: Demonstrate the fundamental problem of digital money without trust
4. **Appreciate Trusted Intermediaries**: Understand why banks exist and how they solve the double-spending problem
5. **Recognize the Cost of Trust**: See how centralization introduces fees, delays, and risks - motivating blockchain

## Section 1: Setup

We'll use only Python's standard library plus `pandas` and `matplotlib` for data handling and visualization.

In [ ]:
# Core imports - all standard library or common packages
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple, Any
import random
import time
import copy

# Data handling and visualization
import pandas as pd
import matplotlib.pyplot as plt

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Matplotlib style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12

print("Libraries imported successfully!")
print(f"\nPandas version: {pd.__version__}")
print(f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Section 2: What is Money?

Before we build our ledger, let's understand what money actually is.

### The Three Functions of Money

1. **Medium of Exchange**: Facilitates trade between parties
2. **Unit of Account**: Provides a common measure of value
3. **Store of Value**: Preserves purchasing power over time

### From Physical to Digital

| Era | Form of Money | Trust Mechanism |
|-----|---------------|----------------|
| Ancient | Commodities (gold, shells) | Physical scarcity |
| Medieval | Coins | Government mint, metal content |
| Modern | Paper currency | Central bank backing |
| Digital | Bank deposits | Financial institutions |
| **Future?** | **Cryptocurrency** | **Cryptographic proof** |

### The Fundamental Problem

Physical money is **self-proving** - if you have a gold coin, it's obvious you have it. But digital information can be copied infinitely. How do we prevent someone from spending the same digital money twice?

This is the **double-spending problem**, and it's the central challenge of digital finance.

In [ ]:
# Demonstration: Physical vs Digital

print("=" * 70)
print("PHYSICAL vs DIGITAL MONEY")
print("=" * 70)

print("\n PHYSICAL MONEY (e.g., Cash)")
print("-" * 40)
print("")
print("  Alice has a $100 bill")
print("  Alice gives the bill to Bob")
print("  Alice NO LONGER has the bill (can't give it to Charlie too!)")
print("")
print("  Physical transfer automatically prevents double-spending.")

print("\n DIGITAL MONEY (without controls)")
print("-" * 40)
print("")
print("  Alice has a file: 'money.txt' containing '$100'")
print("  Alice sends a COPY to Bob")
print("  Alice sends ANOTHER COPY to Charlie")
print("  Alice sends ANOTHER COPY to Dave")
print("")
print("  Alice just spent $100 three times!")
print("")
print("=" * 70)
print("\nThis is the DOUBLE-SPENDING PROBLEM.")
print("Without a solution, digital money is meaningless.")
print("=" * 70)

## Section 3: Simple Ledger Implementation

A **ledger** is the foundation of any financial system. It's simply a record of:
- Who has how much money
- All transactions that have occurred

Let's build a simple ledger from scratch.

In [ ]:
class SimpleLedger:
    """
    A basic ledger that tracks balances and transactions.
    
    This is an UNSECURED ledger - anyone can add transactions.
    We'll see the problems with this approach shortly.
    """
    
    def __init__(self):
        """
        Initialize an empty ledger.
        """
        self.balances: Dict[str, float] = {}  # name -> balance
        self.transactions: List[Dict[str, Any]] = []  # list of all transactions
        self.transaction_counter = 0
    
    def create_account(self, name: str, initial_balance: float = 0.0) -> bool:
        """
        Create a new account with optional initial balance.
        
        Args:
            name: Account holder's name
            initial_balance: Starting balance (default 0)
            
        Returns:
            True if account created, False if already exists
        """
        if name in self.balances:
            print(f"Account '{name}' already exists!")
            return False
        
        self.balances[name] = initial_balance
        
        # Record the initial deposit as a transaction
        if initial_balance > 0:
            self._record_transaction(
                sender="SYSTEM",
                recipient=name,
                amount=initial_balance,
                description="Initial deposit"
            )
        
        print(f"Account created: {name} (Balance: ${initial_balance:.2f})")
        return True
    
    def get_balance(self, name: str) -> Optional[float]:
        """
        Get the current balance of an account.
        
        Args:
            name: Account holder's name
            
        Returns:
            Balance if account exists, None otherwise
        """
        if name not in self.balances:
            print(f"Account '{name}' does not exist!")
            return None
        
        return self.balances[name]
    
    def add_transaction(self, sender: str, recipient: str, amount: float) -> bool:
        """
        Add a transaction to the ledger.
        
        WARNING: This method has NO VALIDATION!
        This is intentional - we'll see why this is a problem.
        
        Args:
            sender: Name of the sender
            recipient: Name of the recipient
            amount: Amount to transfer
            
        Returns:
            True if transaction processed, False otherwise
        """
        # Basic existence checks
        if sender not in self.balances and sender != "SYSTEM":
            print(f"Sender '{sender}' does not exist!")
            return False
        
        if recipient not in self.balances:
            print(f"Recipient '{recipient}' does not exist!")
            return False
        
        if amount <= 0:
            print("Amount must be positive!")
            return False
        
        # Execute the transaction (NO BALANCE CHECK!)
        if sender != "SYSTEM":
            self.balances[sender] -= amount
        self.balances[recipient] += amount
        
        # Record the transaction
        self._record_transaction(sender, recipient, amount)
        
        return True
    
    def _record_transaction(self, sender: str, recipient: str, amount: float, 
                           description: str = "") -> None:
        """
        Internal method to record a transaction.
        """
        self.transaction_counter += 1
        
        tx = {
            'id': self.transaction_counter,
            'timestamp': datetime.now(),
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
            'description': description
        }
        
        self.transactions.append(tx)
    
    def print_balances(self) -> None:
        """
        Print all account balances.
        """
        print("\n" + "=" * 40)
        print("ACCOUNT BALANCES")
        print("=" * 40)
        
        for name, balance in sorted(self.balances.items()):
            status = "" if balance >= 0 else " [NEGATIVE!]"
            print(f"  {name:<15} ${balance:>10.2f}{status}")
        
        total = sum(self.balances.values())
        print("-" * 40)
        print(f"  {'TOTAL':<15} ${total:>10.2f}")
        print("=" * 40 + "\n")
    
    def print_transactions(self) -> None:
        """
        Print all transactions.
        """
        print("\n" + "=" * 70)
        print("TRANSACTION HISTORY")
        print("=" * 70)
        
        if not self.transactions:
            print("  No transactions recorded.")
        else:
            for tx in self.transactions:
                time_str = tx['timestamp'].strftime('%H:%M:%S')
                desc = f" ({tx['description']})" if tx['description'] else ""
                print(f"  #{tx['id']:03d} [{time_str}] {tx['sender']} -> {tx['recipient']}: ${tx['amount']:.2f}{desc}")
        
        print("=" * 70 + "\n")

In [ ]:
# Let's test our simple ledger

print("CREATING A SIMPLE LEDGER\n")

ledger = SimpleLedger()

# Create accounts with initial balances
ledger.create_account("Alice", 100.0)
ledger.create_account("Bob", 50.0)
ledger.create_account("Charlie", 25.0)

# Show initial state
ledger.print_balances()

# Make some transactions
print("\nProcessing transactions...\n")

print("Transaction 1: Alice sends $30 to Bob")
ledger.add_transaction("Alice", "Bob", 30.0)

print("Transaction 2: Bob sends $20 to Charlie")
ledger.add_transaction("Bob", "Charlie", 20.0)

print("Transaction 3: Charlie sends $10 to Alice")
ledger.add_transaction("Charlie", "Alice", 10.0)

# Show final state
ledger.print_balances()
ledger.print_transactions()

### The Ledger Seems to Work!

Our simple ledger successfully:
- Creates accounts with initial balances
- Records transactions
- Updates balances correctly

But there's a **critical flaw**. Can you spot it?

## Section 4: The Double-Spending Problem

Our simple ledger has a **fatal flaw**: it doesn't check if the sender has sufficient balance!

Let's demonstrate the double-spending problem:

In [ ]:
print("=" * 70)
print("DOUBLE-SPENDING ATTACK DEMONSTRATION")
print("=" * 70)

# Create a fresh ledger
vulnerable_ledger = SimpleLedger()

# Alice has $100
vulnerable_ledger.create_account("Alice", 100.0)
vulnerable_ledger.create_account("Bob", 0.0)
vulnerable_ledger.create_account("Charlie", 0.0)
vulnerable_ledger.create_account("Dave", 0.0)

print("\nInitial balances:")
vulnerable_ledger.print_balances()

print("\n" + "=" * 70)
print("ALICE ATTEMPTS TO DOUBLE-SPEND!")
print("=" * 70)

print("\nAlice sends $100 to Bob...")
vulnerable_ledger.add_transaction("Alice", "Bob", 100.0)

print("Alice sends $100 to Charlie (same $100!)...")
vulnerable_ledger.add_transaction("Alice", "Charlie", 100.0)

print("Alice sends $100 to Dave (same $100 again!)...")
vulnerable_ledger.add_transaction("Alice", "Dave", 100.0)

print("\nFinal balances after attack:")
vulnerable_ledger.print_balances()

In [ ]:
# Let's visualize what went wrong

print("\n" + "=" * 70)
print("ANALYSIS OF THE ATTACK")
print("=" * 70)

print("\nAlice started with:           $100.00")
print("Alice sent to Bob:            $100.00")
print("Alice sent to Charlie:        $100.00")
print("Alice sent to Dave:           $100.00")
print("-" * 40)
print("Total Alice 'spent':          $300.00")
print("\nBut Alice only HAD:           $100.00!")

print("\n" + "=" * 70)
print("MONEY WAS CREATED OUT OF THIN AIR!")
print("=" * 70)

total_initial = 100.0 + 0.0 + 0.0 + 0.0
total_final = sum(vulnerable_ledger.balances.values())

print(f"\nTotal money in system initially: ${total_initial:.2f}")
print(f"Total money in system now:       ${total_final:.2f}")
print(f"Money 'created' by attack:       ${total_final - total_initial:.2f}")

print("\n" + "=" * 70)
print("\nThis is why we need TRUST or VERIFICATION!")
print("Without it, digital money is worthless.")
print("=" * 70)

### Why Does This Matter?

In the physical world, double-spending is impossible:
- You can't give the same $100 bill to two people
- The laws of physics prevent copying physical objects

In the digital world:
- Files can be copied infinitely at zero cost
- Network delays mean different parties see different states
- Without verification, anyone can claim to have any amount

**The double-spending problem is THE fundamental challenge of digital money.**

## Section 5: Adding a Trusted Authority (The Bank)

The traditional solution to double-spending is a **trusted intermediary** - the bank.

The bank:
- Maintains the authoritative ledger
- Validates all transactions
- Prevents double-spending by checking balances

Let's implement a proper Bank class:

In [ ]:
class Bank:
    """
    A trusted bank that validates all transactions.
    
    This solves the double-spending problem by:
    1. Maintaining the SINGLE authoritative ledger
    2. Validating sufficient balance before transfers
    3. Processing transactions atomically
    """
    
    def __init__(self, name: str = "Central Bank"):
        """
        Initialize the bank.
        
        Args:
            name: Name of the bank
        """
        self.name = name
        self.balances: Dict[str, float] = {}
        self.transactions: List[Dict[str, Any]] = []
        self.transaction_counter = 0
        self.failed_transactions: List[Dict[str, Any]] = []
        
        # Bank fees and processing time
        self.transfer_fee_percent = 1.0  # 1% fee
        self.minimum_fee = 0.50  # Minimum $0.50 fee
        self.processing_delay = 0.5  # Simulated processing time in seconds
        
        print(f"Bank '{self.name}' initialized.")
        print(f"  Transfer fee: {self.transfer_fee_percent}% (minimum ${self.minimum_fee:.2f})")
    
    def create_account(self, name: str, initial_balance: float = 0.0) -> bool:
        """
        Create a new bank account.
        
        Args:
            name: Account holder's name
            initial_balance: Starting balance
            
        Returns:
            True if account created successfully
        """
        if name in self.balances:
            print(f"  [REJECTED] Account '{name}' already exists!")
            return False
        
        if initial_balance < 0:
            print(f"  [REJECTED] Initial balance cannot be negative!")
            return False
        
        self.balances[name] = initial_balance
        
        if initial_balance > 0:
            self._record_transaction(
                sender="DEPOSIT",
                recipient=name,
                amount=initial_balance,
                fee=0.0,
                status="COMPLETED",
                description="Initial deposit"
            )
        
        print(f"  [APPROVED] Account '{name}' created with ${initial_balance:.2f}")
        return True
    
    def get_balance(self, name: str) -> Optional[float]:
        """
        Get current account balance.
        """
        return self.balances.get(name)
    
    def calculate_fee(self, amount: float) -> float:
        """
        Calculate the transaction fee.
        
        Args:
            amount: Transaction amount
            
        Returns:
            Fee amount
        """
        fee = amount * (self.transfer_fee_percent / 100)
        return max(fee, self.minimum_fee)
    
    def transfer(self, sender: str, recipient: str, amount: float) -> Dict[str, Any]:
        """
        Transfer money between accounts with full validation.
        
        Args:
            sender: Sender's name
            recipient: Recipient's name
            amount: Amount to transfer
            
        Returns:
            Dictionary with transaction result
        """
        print(f"\n  Processing: {sender} -> {recipient}: ${amount:.2f}")
        
        # Simulate processing delay
        time.sleep(self.processing_delay)
        
        # Validation 1: Check sender exists
        if sender not in self.balances:
            return self._reject_transaction(
                sender, recipient, amount, 
                f"Sender account '{sender}' does not exist"
            )
        
        # Validation 2: Check recipient exists
        if recipient not in self.balances:
            return self._reject_transaction(
                sender, recipient, amount,
                f"Recipient account '{recipient}' does not exist"
            )
        
        # Validation 3: Check amount is positive
        if amount <= 0:
            return self._reject_transaction(
                sender, recipient, amount,
                "Transfer amount must be positive"
            )
        
        # Validation 4: Calculate fee and total cost
        fee = self.calculate_fee(amount)
        total_cost = amount + fee
        
        # Validation 5: CHECK SUFFICIENT BALANCE (prevents double-spending!)
        if self.balances[sender] < total_cost:
            return self._reject_transaction(
                sender, recipient, amount,
                f"Insufficient funds: has ${self.balances[sender]:.2f}, needs ${total_cost:.2f} (including ${fee:.2f} fee)"
            )
        
        # All validations passed - execute the transfer
        self.balances[sender] -= total_cost
        self.balances[recipient] += amount
        
        # Record the transaction
        tx = self._record_transaction(
            sender=sender,
            recipient=recipient,
            amount=amount,
            fee=fee,
            status="COMPLETED"
        )
        
        print(f"  [APPROVED] Transfer complete. Fee: ${fee:.2f}")
        
        return {
            'success': True,
            'transaction_id': tx['id'],
            'amount': amount,
            'fee': fee,
            'sender_balance': self.balances[sender],
            'recipient_balance': self.balances[recipient]
        }
    
    def _reject_transaction(self, sender: str, recipient: str, 
                           amount: float, reason: str) -> Dict[str, Any]:
        """
        Record and return a rejected transaction.
        """
        tx = self._record_transaction(
            sender=sender,
            recipient=recipient,
            amount=amount,
            fee=0.0,
            status="REJECTED",
            description=reason
        )
        
        self.failed_transactions.append(tx)
        
        print(f"  [REJECTED] {reason}")
        
        return {
            'success': False,
            'transaction_id': tx['id'],
            'reason': reason
        }
    
    def _record_transaction(self, sender: str, recipient: str, amount: float,
                           fee: float, status: str, description: str = "") -> Dict[str, Any]:
        """
        Record a transaction in the ledger.
        """
        self.transaction_counter += 1
        
        tx = {
            'id': self.transaction_counter,
            'timestamp': datetime.now(),
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
            'fee': fee,
            'status': status,
            'description': description
        }
        
        self.transactions.append(tx)
        return tx
    
    def print_balances(self) -> None:
        """
        Print all account balances.
        """
        print(f"\n{'=' * 50}")
        print(f"{self.name.upper()} - ACCOUNT BALANCES")
        print(f"{'=' * 50}")
        
        for name, balance in sorted(self.balances.items()):
            print(f"  {name:<20} ${balance:>12.2f}")
        
        total = sum(self.balances.values())
        print(f"{'-' * 50}")
        print(f"  {'TOTAL':<20} ${total:>12.2f}")
        print(f"{'=' * 50}\n")
    
    def print_transactions(self, show_rejected: bool = True) -> None:
        """
        Print transaction history.
        """
        print(f"\n{'=' * 80}")
        print(f"{self.name.upper()} - TRANSACTION HISTORY")
        print(f"{'=' * 80}")
        
        for tx in self.transactions:
            if tx['status'] == 'REJECTED' and not show_rejected:
                continue
            
            time_str = tx['timestamp'].strftime('%H:%M:%S')
            status_symbol = '[OK]' if tx['status'] == 'COMPLETED' else '[X]'
            fee_str = f" (fee: ${tx['fee']:.2f})" if tx['fee'] > 0 else ""
            
            print(f"  #{tx['id']:03d} {status_symbol} [{time_str}] {tx['sender']} -> {tx['recipient']}: ${tx['amount']:.2f}{fee_str}")
            
            if tx['description']:
                print(f"       Note: {tx['description']}")
        
        print(f"{'=' * 80}")
        print(f"  Total transactions: {len(self.transactions)}")
        print(f"  Completed: {len(self.transactions) - len(self.failed_transactions)}")
        print(f"  Rejected: {len(self.failed_transactions)}")
        print(f"{'=' * 80}\n")

In [ ]:
# Test the Bank - it should PREVENT double-spending!

print("=" * 70)
print("TESTING THE BANK (Trusted Intermediary)")
print("=" * 70)

bank = Bank("First National Bank")

print("\nCreating accounts...")
bank.create_account("Alice", 100.0)
bank.create_account("Bob", 0.0)
bank.create_account("Charlie", 0.0)
bank.create_account("Dave", 0.0)

bank.print_balances()

In [ ]:
# Now let's try the SAME double-spending attack

print("=" * 70)
print("ATTEMPTING DOUBLE-SPENDING ATTACK")
print("=" * 70)

print("\nAlice tries to send $100 to Bob...")
result1 = bank.transfer("Alice", "Bob", 100.0)

print("\nAlice tries to send $100 to Charlie (same $100!)...")
result2 = bank.transfer("Alice", "Charlie", 100.0)

print("\nAlice tries to send $100 to Dave (same $100 again!)...")
result3 = bank.transfer("Alice", "Dave", 100.0)

In [ ]:
# Check the results

print("\n" + "=" * 70)
print("RESULTS: DOUBLE-SPENDING PREVENTED!")
print("=" * 70)

bank.print_balances()

print("\n ATTACK RESULTS:")
print("-" * 40)
print(f"  Transaction 1 (Alice -> Bob):     {'SUCCESS' if result1['success'] else 'BLOCKED'}")
print(f"  Transaction 2 (Alice -> Charlie): {'SUCCESS' if result2['success'] else 'BLOCKED'}")
print(f"  Transaction 3 (Alice -> Dave):    {'SUCCESS' if result3['success'] else 'BLOCKED'}")

# Verify total money is preserved
total = sum(bank.balances.values())
initial_total = 100.0  # Only Alice had money initially
fees_collected = sum(tx['fee'] for tx in bank.transactions if tx['status'] == 'COMPLETED' and tx['fee'] > 0)

print(f"\n MONEY SUPPLY:")
print("-" * 40)
print(f"  Initial total:     ${initial_total:.2f}")
print(f"  Current total:     ${total:.2f}")
print(f"  Fees collected:    ${fees_collected:.2f}")
print(f"  Money created:     ${total + fees_collected - initial_total:.2f} (should be zero!)")

print("\n" + "=" * 70)
print("The bank successfully prevented Alice from double-spending!")
print("=" * 70)

In [ ]:
# View the full transaction history
bank.print_transactions()

### The Bank Solution Works!

The bank successfully prevents double-spending by:

1. **Single Source of Truth**: Only the bank's ledger matters
2. **Balance Validation**: Every transaction is checked against available funds
3. **Atomic Execution**: Transfers complete fully or not at all
4. **Sequential Processing**: One transaction at a time, no race conditions

This is how traditional banking has worked for centuries. **But there's a cost...**

## Section 6: Visualizing Transaction History

Let's create a visualization showing how balances change over time with our bank transactions.

In [ ]:
def simulate_transaction_history(num_transactions: int = 20) -> Bank:
    """
    Simulate a series of random transactions to visualize.
    
    Args:
        num_transactions: Number of transactions to simulate
        
    Returns:
        Bank with transaction history
    """
    # Create bank with no delay for faster simulation
    bank = Bank("Visualization Bank")
    bank.processing_delay = 0  # No delay for simulation
    bank.transfer_fee_percent = 0.5  # Lower fees for simulation
    bank.minimum_fee = 0.10
    
    # Create accounts
    accounts = ["Alice", "Bob", "Charlie", "Dave"]
    for account in accounts:
        bank.create_account(account, random.randint(50, 150))
    
    # Simulate random transactions
    print(f"\nSimulating {num_transactions} transactions...\n")
    
    for i in range(num_transactions):
        sender = random.choice(accounts)
        recipient = random.choice([a for a in accounts if a != sender])
        
        # Random amount between $5 and $30
        amount = random.uniform(5, 30)
        
        bank.transfer(sender, recipient, amount)
    
    return bank

# Run simulation
sim_bank = simulate_transaction_history(20)

In [ ]:
def plot_balance_history(bank: Bank) -> None:
    """
    Plot how balances changed over time.
    """
    # Get unique accounts (excluding DEPOSIT and SYSTEM)
    accounts = sorted([name for name in bank.balances.keys()])
    
    # Build balance history
    history = {account: [0.0] for account in accounts}
    
    # Track running balances
    running_balance = {account: 0.0 for account in accounts}
    
    for tx in bank.transactions:
        if tx['status'] != 'COMPLETED':
            continue
        
        sender = tx['sender']
        recipient = tx['recipient']
        amount = tx['amount']
        fee = tx['fee']
        
        # Update balances
        if sender == 'DEPOSIT':
            running_balance[recipient] += amount
        elif sender in accounts:
            running_balance[sender] -= (amount + fee)
            if recipient in accounts:
                running_balance[recipient] += amount
        
        # Record state after this transaction
        for account in accounts:
            history[account].append(running_balance[account])
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 6))
    
    colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D']
    
    for i, account in enumerate(accounts):
        ax.plot(history[account], label=account, linewidth=2, 
                color=colors[i % len(colors)], marker='o', markersize=4)
    
    ax.set_xlabel('Transaction Number', fontsize=12)
    ax.set_ylabel('Balance ($)', fontsize=12)
    ax.set_title('Account Balance History Over Time', fontsize=14, fontweight='bold')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    
    # Add horizontal line at y=0
    ax.axhline(y=0, color='red', linestyle='--', alpha=0.5, label='Zero balance')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\nBalance Summary:")
    print("-" * 50)
    for account in accounts:
        initial = history[account][1] if len(history[account]) > 1 else 0
        final = history[account][-1]
        change = final - initial
        print(f"  {account}: ${initial:.2f} -> ${final:.2f} (change: ${change:+.2f})")

# Plot the balance history
plot_balance_history(sim_bank)

In [ ]:
def plot_transaction_analysis(bank: Bank) -> None:
    """
    Create analysis charts for transactions.
    """
    # Separate completed and rejected transactions
    completed = [tx for tx in bank.transactions if tx['status'] == 'COMPLETED' and tx['sender'] != 'DEPOSIT']
    rejected = [tx for tx in bank.transactions if tx['status'] == 'REJECTED']
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Chart 1: Transaction Status
    ax1 = axes[0]
    status_counts = [len(completed), len(rejected)]
    status_labels = ['Completed', 'Rejected']
    colors = ['#2E86AB', '#C73E1D']
    
    ax1.pie(status_counts, labels=status_labels, colors=colors, autopct='%1.1f%%',
            startangle=90, explode=(0.05, 0.05))
    ax1.set_title('Transaction Status', fontweight='bold')
    
    # Chart 2: Fee Distribution
    ax2 = axes[1]
    if completed:
        fees = [tx['fee'] for tx in completed]
        ax2.hist(fees, bins=10, color='#A23B72', edgecolor='black', alpha=0.7)
        ax2.set_xlabel('Fee Amount ($)')
        ax2.set_ylabel('Number of Transactions')
        ax2.set_title('Distribution of Transaction Fees', fontweight='bold')
        ax2.axvline(x=sum(fees)/len(fees), color='red', linestyle='--', 
                    label=f'Mean: ${sum(fees)/len(fees):.2f}')
        ax2.legend()
    
    # Chart 3: Transaction Amounts
    ax3 = axes[2]
    if completed:
        amounts = [tx['amount'] for tx in completed]
        ax3.hist(amounts, bins=10, color='#F18F01', edgecolor='black', alpha=0.7)
        ax3.set_xlabel('Transaction Amount ($)')
        ax3.set_ylabel('Number of Transactions')
        ax3.set_title('Distribution of Transaction Amounts', fontweight='bold')
        ax3.axvline(x=sum(amounts)/len(amounts), color='red', linestyle='--',
                    label=f'Mean: ${sum(amounts)/len(amounts):.2f}')
        ax3.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print("\nTransaction Statistics:")
    print("=" * 50)
    print(f"  Total transactions attempted: {len(bank.transactions)}")
    print(f"  Completed: {len(completed)}")
    print(f"  Rejected: {len(rejected)} (prevented double-spending!)")
    
    if completed:
        total_volume = sum(tx['amount'] for tx in completed)
        total_fees = sum(tx['fee'] for tx in completed)
        print(f"\n  Total volume: ${total_volume:.2f}")
        print(f"  Total fees: ${total_fees:.2f}")
        print(f"  Average fee rate: {(total_fees/total_volume)*100:.2f}%")

# Create analysis charts
plot_transaction_analysis(sim_bank)

## Section 7: The Cost of Trust

Our bank solves double-spending, but at what cost? Let's examine the trade-offs of centralized trust.

In [ ]:
print("=" * 70)
print("THE COST OF CENTRALIZED TRUST")
print("=" * 70)

print("""
Our bank successfully prevents double-spending, but introduces:

1. FEES
   - Transaction fees (1-3% typically)
   - Account maintenance fees
   - International transfer fees (3-5%)
   - Currency conversion fees
   
2. DELAYS
   - Domestic transfers: 1-3 business days
   - International transfers: 3-5 business days
   - Business hours only
   - Weekend/holiday closures
   
3. ACCESS RESTRICTIONS
   - Must have government ID
   - Credit checks required
   - Minimum balance requirements
   - Geographic restrictions
   - 1.4 billion adults unbanked globally
   
4. CENSORSHIP RISK
   - Accounts can be frozen
   - Transactions can be blocked
   - Governments can seize funds
   - Banks can refuse service
   
5. COUNTERPARTY RISK
   - Bank can fail (2008 financial crisis)
   - Deposits are only insured up to limits
   - Fractional reserve banking
   - Systemic risk in banking system
   
6. PRIVACY CONCERNS
   - All transactions visible to bank
   - Data shared with governments
   - Data breaches expose customer info
   - Financial surveillance
""")

print("=" * 70)

In [ ]:
def demonstrate_trust_costs() -> None:
    """
    Demonstrate the various costs of using a trusted intermediary.
    """
    print("\n" + "=" * 70)
    print("REAL-WORLD COST COMPARISON")
    print("=" * 70)
    
    # Example: International transfer of $1000
    amount = 1000.0
    
    print(f"\nScenario: Sending ${amount:.2f} from USA to Europe\n")
    
    methods = [
        {
            'name': 'Traditional Bank (Wire)',
            'fee_percent': 0.5,
            'fixed_fee': 45.0,
            'fx_markup': 3.0,
            'time': '3-5 business days'
        },
        {
            'name': 'PayPal',
            'fee_percent': 4.4,
            'fixed_fee': 0.30,
            'fx_markup': 2.5,
            'time': 'Instant to 1 day'
        },
        {
            'name': 'Western Union',
            'fee_percent': 0.0,
            'fixed_fee': 25.0,
            'fx_markup': 4.0,
            'time': 'Minutes to 5 days'
        },
        {
            'name': 'Wise (TransferWise)',
            'fee_percent': 0.5,
            'fixed_fee': 3.0,
            'fx_markup': 0.5,
            'time': '1-2 business days'
        },
        {
            'name': 'Bitcoin (on-chain)',
            'fee_percent': 0.0,
            'fixed_fee': 2.0,  # Average network fee
            'fx_markup': 0.0,
            'time': '10-60 minutes'
        },
        {
            'name': 'Stablecoin (USDC)',
            'fee_percent': 0.0,
            'fixed_fee': 1.0,  # Network fee
            'fx_markup': 0.0,
            'time': '15 seconds - 15 minutes'
        }
    ]
    
    print(f"{'Method':<25} {'Fee':<15} {'FX Cost':<15} {'Total Cost':<15} {'Time'}")
    print("-" * 90)
    
    for method in methods:
        fee = (amount * method['fee_percent'] / 100) + method['fixed_fee']
        fx_cost = amount * method['fx_markup'] / 100
        total = fee + fx_cost
        
        print(f"{method['name']:<25} ${fee:<14.2f} ${fx_cost:<14.2f} ${total:<14.2f} {method['time']}")
    
    print("\n" + "-" * 90)
    print("Note: Crypto costs don't include exchange fees for on/off ramps")
    print("      Actual costs vary based on network congestion and market conditions")
    print("=" * 70)

demonstrate_trust_costs()

In [ ]:
def visualize_trust_tradeoffs() -> None:
    """
    Visualize the trade-offs between different money systems.
    """
    # Define characteristics (0-10 scale)
    systems = {
        'Cash': {
            'Speed': 10,
            'Cost': 10,
            'Privacy': 9,
            'Scalability': 2,
            'Security': 4,
            'Accessibility': 8
        },
        'Bank Transfer': {
            'Speed': 4,
            'Cost': 5,
            'Privacy': 3,
            'Scalability': 8,
            'Security': 7,
            'Accessibility': 5
        },
        'Bitcoin': {
            'Speed': 6,
            'Cost': 7,
            'Privacy': 6,
            'Scalability': 4,
            'Security': 9,
            'Accessibility': 6
        },
        'Stablecoin': {
            'Speed': 9,
            'Cost': 8,
            'Privacy': 5,
            'Scalability': 7,
            'Security': 7,
            'Accessibility': 7
        }
    }
    
    categories = list(systems['Cash'].keys())
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = range(len(categories))
    width = 0.2
    colors = ['#2E86AB', '#A23B72', '#F18F01', '#48A14D']
    
    for i, (system, values) in enumerate(systems.items()):
        scores = [values[cat] for cat in categories]
        ax.bar([xi + i * width for xi in x], scores, width, label=system, color=colors[i])
    
    ax.set_xlabel('Characteristic', fontsize=12)
    ax.set_ylabel('Score (0-10)', fontsize=12)
    ax.set_title('Comparison of Money Systems', fontsize=14, fontweight='bold')
    ax.set_xticks([xi + 1.5 * width for xi in x])
    ax.set_xticklabels(categories)
    ax.legend()
    ax.set_ylim(0, 11)
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print("\nKey Observations:")
    print("-" * 50)
    print("- Cash: Fast and private, but doesn't scale and hard to secure")
    print("- Bank Transfer: Secure and scalable, but slow, costly, and excludes many")
    print("- Bitcoin: Very secure and censorship-resistant, but can be slow")
    print("- Stablecoins: Balance of speed, cost, and accessibility")
    print("\nNo system is perfect - blockchain aims to optimize different trade-offs!")

visualize_trust_tradeoffs()

### The Blockchain Promise

**What if we could have the security of banks WITHOUT the central authority?**

This is the fundamental promise of blockchain:

| Traditional Finance | Blockchain Finance |
|--------------------|--------------------|  
| Trust the bank | Trust the math |
| Bank validates | Network validates |
| Bank stores ledger | Everyone stores ledger |
| Bank can censor | No single point of censorship |
| Bank hours | 24/7/365 |
| Geographic limits | Global by default |
| Identity required | Pseudonymous |

In the next notebooks, we'll explore how cryptography and distributed systems make this possible.

## Section 8: Challenge Exercises

Test your understanding with these challenges!

### Challenge 1: Implement Balance Validation

Modify the `SimpleLedger` class to add balance checking. Create a new method `add_transaction_safe()` that rejects transactions when the sender has insufficient funds.

In [ ]:
# YOUR TURN: Complete this challenge!

class SafeLedger(SimpleLedger):
    """
    A ledger that validates balances before transactions.
    """
    
    def add_transaction_safe(self, sender: str, recipient: str, amount: float) -> bool:
        """
        Add a transaction with balance validation.
        
        TODO: Implement this method!
        
        Requirements:
        1. Check that sender exists
        2. Check that recipient exists
        3. Check that amount is positive
        4. Check that sender has sufficient balance
        5. If all checks pass, execute the transaction
        6. Return True if successful, False otherwise
        """
        # Your code here!
        pass

# Test your implementation
print("Testing SafeLedger...\n")

safe = SafeLedger()
safe.create_account("Alice", 100.0)
safe.create_account("Bob", 0.0)

# This should succeed
print("\nTest 1: Alice sends $50 to Bob (should succeed)")
# result1 = safe.add_transaction_safe("Alice", "Bob", 50.0)
# print(f"Result: {result1}")

# This should fail (insufficient funds)
print("\nTest 2: Alice sends $100 to Bob (should fail - only $50 left)")
# result2 = safe.add_transaction_safe("Alice", "Bob", 100.0)
# print(f"Result: {result2}")

# safe.print_balances()

print("\n[Challenge 1: Implement the add_transaction_safe method above!]")

### Challenge 2: Simulate a Bank Run

Banks use fractional reserve banking - they only keep a fraction of deposits on hand. What happens when everyone tries to withdraw at once?

In [ ]:
# YOUR TURN: Complete this challenge!

class FractionalReserveBank:
    """
    A bank that practices fractional reserve banking.
    
    The bank only keeps a fraction of deposits as reserves,
    lending the rest out to earn interest.
    """
    
    def __init__(self, name: str, reserve_ratio: float = 0.10):
        """
        Initialize the bank.
        
        Args:
            name: Bank name
            reserve_ratio: Fraction of deposits to keep as reserves (e.g., 0.10 = 10%)
        """
        self.name = name
        self.reserve_ratio = reserve_ratio
        self.deposits: Dict[str, float] = {}  # What customers think they have
        self.reserves: float = 0.0  # Actual cash on hand
        self.loans_outstanding: float = 0.0  # Money lent out
        
        print(f"Bank '{name}' created with {reserve_ratio*100:.0f}% reserve ratio")
    
    def deposit(self, customer: str, amount: float) -> None:
        """
        Customer deposits money.
        """
        if customer not in self.deposits:
            self.deposits[customer] = 0.0
        
        self.deposits[customer] += amount
        
        # Keep only reserve_ratio in reserves, "lend" the rest
        reserve_amount = amount * self.reserve_ratio
        loan_amount = amount - reserve_amount
        
        self.reserves += reserve_amount
        self.loans_outstanding += loan_amount
        
        print(f"  {customer} deposits ${amount:.2f}")
        print(f"    -> Reserves: +${reserve_amount:.2f}, Loans: +${loan_amount:.2f}")
    
    def withdraw(self, customer: str, amount: float) -> bool:
        """
        Customer attempts to withdraw money.
        
        TODO: Implement withdrawal logic!
        
        Returns:
            True if withdrawal successful, False if bank doesn't have funds
        """
        # Your code here!
        # Hint: Check if reserves are sufficient, not just the customer's balance
        pass
    
    def print_status(self) -> None:
        """
        Print bank's financial status.
        """
        total_deposits = sum(self.deposits.values())
        
        print(f"\n{'=' * 50}")
        print(f"BANK STATUS: {self.name}")
        print(f"{'=' * 50}")
        print(f"  Total customer deposits: ${total_deposits:.2f}")
        print(f"  Reserves (cash on hand): ${self.reserves:.2f}")
        print(f"  Loans outstanding:       ${self.loans_outstanding:.2f}")
        print(f"  Reserve ratio:           {(self.reserves/total_deposits)*100:.1f}%" if total_deposits > 0 else "  Reserve ratio: N/A")
        print(f"{'=' * 50}\n")

# Test: Simulate a bank run
print("\n" + "=" * 70)
print("BANK RUN SIMULATION")
print("=" * 70)

fractional_bank = FractionalReserveBank("Risky Bank", reserve_ratio=0.10)

# Multiple customers deposit money
print("\nCustomers deposit money:")
for i in range(5):
    fractional_bank.deposit(f"Customer_{i+1}", 1000.0)

fractional_bank.print_status()

print("\n" + "=" * 70)
print("PANIC! Everyone tries to withdraw at once!")
print("=" * 70)

# TODO: Implement withdrawal and simulate everyone trying to withdraw
# for i in range(5):
#     result = fractional_bank.withdraw(f"Customer_{i+1}", 1000.0)
#     print(f"Customer_{i+1} withdrawal: {'SUCCESS' if result else 'FAILED'}")

print("\n[Challenge 2: Implement the withdraw method and run the simulation!]")

### Challenge 3: Calculate the True Cost of a Transfer

Build a function that calculates the total cost of transferring money internationally, including fees, exchange rate markups, and time value of money.

In [ ]:
# YOUR TURN: Complete this challenge!

def calculate_transfer_cost(
    amount: float,
    fee_percent: float,
    fixed_fee: float,
    fx_markup_percent: float,
    days_delay: int,
    annual_interest_rate: float = 0.05
) -> Dict[str, float]:
    """
    Calculate the total true cost of a money transfer.
    
    Args:
        amount: Amount to transfer
        fee_percent: Percentage fee charged
        fixed_fee: Fixed fee charged
        fx_markup_percent: Exchange rate markup percentage
        days_delay: Number of days until money arrives
        annual_interest_rate: Opportunity cost rate (default 5%)
        
    Returns:
        Dictionary with breakdown of costs
        
    TODO: Calculate:
    1. Percentage fee
    2. Fixed fee (already given)
    3. FX markup cost
    4. Opportunity cost (interest lost during delay)
    5. Total cost
    6. Effective fee percentage (total cost / amount)
    """
    # Your code here!
    
    return {
        'amount': amount,
        'percentage_fee': 0.0,  # Calculate this
        'fixed_fee': fixed_fee,
        'fx_cost': 0.0,  # Calculate this
        'opportunity_cost': 0.0,  # Calculate this
        'total_cost': 0.0,  # Calculate this
        'effective_rate': 0.0  # Calculate this
    }

# Test your implementation
print("Transfer Cost Analysis")
print("=" * 50)

# Example: $1000 international wire transfer
# result = calculate_transfer_cost(
#     amount=1000.0,
#     fee_percent=0.5,
#     fixed_fee=45.0,
#     fx_markup_percent=3.0,
#     days_delay=5
# )

# print(f"\nTransfer Amount: ${result['amount']:.2f}")
# print(f"Percentage Fee:  ${result['percentage_fee']:.2f}")
# print(f"Fixed Fee:       ${result['fixed_fee']:.2f}")
# print(f"FX Markup Cost:  ${result['fx_cost']:.2f}")
# print(f"Opportunity Cost: ${result['opportunity_cost']:.2f}")
# print(f"-" * 30)
# print(f"Total Cost:      ${result['total_cost']:.2f}")
# print(f"Effective Rate:  {result['effective_rate']*100:.2f}%")

print("\n[Challenge 3: Implement the calculate_transfer_cost function!]")

## Summary

In this notebook, you learned:

### Key Concepts

1. **Ledgers are Fundamental**
   - Every financial system is built on ledgers
   - Ledgers track who owns what and record all transfers
   - The integrity of the ledger is critical to trust in money

2. **The Double-Spending Problem**
   - Digital information can be copied, unlike physical cash
   - Without validation, digital money can be spent multiple times
   - This is THE fundamental challenge of digital finance

3. **Trusted Intermediaries (Banks)**
   - Banks solve double-spending by maintaining the authoritative ledger
   - Every transaction is validated before execution
   - This centralized trust has worked for centuries

4. **The Cost of Centralized Trust**
   - Fees: 1-5% or more for transfers
   - Delays: Days for international transfers
   - Exclusion: 1.4 billion adults unbanked
   - Censorship: Accounts can be frozen
   - Counterparty risk: Banks can fail
   - Privacy: All transactions visible to bank

### Why This Matters for Blockchain

Blockchain technology promises to solve double-spending WITHOUT centralized trust:

- **Distributed Ledger**: Many copies, no single point of failure
- **Cryptographic Proof**: Math replaces institutional trust
- **Consensus Mechanism**: Network agrees on transaction validity
- **Immutability**: Once recorded, transactions cannot be altered
- **Permissionless**: Anyone can participate

### Next Steps

In the following notebooks, we'll explore:
- Cryptographic hash functions (the building blocks of blockchain)
- Digital signatures (how to prove ownership without revealing secrets)
- Distributed consensus (how networks agree without central authority)
- Smart contracts (programmable money)

### Further Reading

- [Bitcoin Whitepaper](https://bitcoin.org/bitcoin.pdf) - Satoshi Nakamoto's original solution to double-spending
- [The Ascent of Money](https://www.amazon.com/Ascent-Money-Financial-History-World/dp/0143116177) - Niall Ferguson's history of financial systems
- [Money: The Unauthorized Biography](https://www.amazon.com/Money-Unauthorized-Biography-Felix-Martin/dp/0307962431) - Felix Martin on what money really is
- [World Bank Global Findex](https://globalfindex.worldbank.org/) - Data on financial inclusion